# 03. Spark-wind-analysis

In [1]:
%config IPCompleter.greedy=True

In [2]:
# spark initialisation, needs to be done before later imports
import findspark
findspark.init()

import pyspark
sc=pyspark.SparkContext.getOrCreate()
sc.setLogLevel('ERROR')

#where the data is stored
datafiles='*.csv'



#initialisation of spark sql session
from pyspark.sql import Row, SparkSession
spark = SparkSession.builder.getOrCreate()


# read the wind data from CSV files
df = spark.read.csv(datafiles, header=True)

# show the top 5 rows
df.show(5)



+----------+--------------------+--------------+----------------+-----------------+---------------------+---------------------------+------------------+-------------------------+----------------------------+
|Station_ID|        Station_Name|Location_Label|Interval_Minutes|Interval_End_Time|Wind_Velocity_Mtr_Sec|Wind_Direction_Variance_Deg|Wind_Direction_Deg|Ambient_Temperature_Deg_C|Global_Horizontal_Irradiance|
+----------+--------------------+--------------+----------------+-----------------+---------------------+---------------------------+------------------+-------------------------+----------------------------+
|      SF15|Warnerville Switc...|   Warnerville|               5| 2015-01-5? 00:05|                1.628|                        8.1|             148.5|                     0.92|                       0.061|
|      SF15|Warnerville Switc...|   Warnerville|               5| 2015-01-5? 00:10|                1.519|                        9.4|             151.1|                

In [3]:
winds=df.rdd
mapped = winds.map(lambda s: (s.Station_ID, s.Wind_Velocity_Mtr_Sec))


This code will display any values which are not strings (and count how many are strings)

In [5]:
count=0
total=0
for k,v in mapped.collect():
    total+=1
    if isinstance(v,str):
        count+=1
    else:
        print(k,v)
        
print(count,total)

SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None
SF04 None


In [6]:
#filter nulls
filtered =mapped.filter(lambda s:s[1]!=None)

In [7]:
#clean nulls
def clean(value):
    try:
        return(float(value))
    except:
        return 0
    
cleaned = mapped.map(lambda s:(s[0], clean(s[1])))



In [ ]:
#count=0
#total=0
#for k,v in cleaned.collect():
#    print(k,v)
#    break
        
#print(count,total)

In [8]:
maxes = filtered.reduceByKey(lambda a, b: a if (a>b) else b)

In [9]:
for k,v in maxes.collect():print(k,v)

SF17 5.767
SF36 9.71
SF04 9.92
SF18 9.85
SF15 7.92
SF37 7.079


In [10]:
print(maxes.collectAsMap())

{'SF17': '5.767', 'SF36': '9.71', 'SF04': '9.92', 'SF18': '9.85', 'SF15': '7.92', 'SF37': '7.079'}


In [11]:
print(maxes.collectAsMap()['SF04'])

9.92


## Part 2
Now lets have a go at using SQL rather than MR.

In [12]:
df.registerTempTable('wind')

In [13]:
spark.sql("SELECT Station_ID, avg(Wind_Velocity_Mtr_Sec) as avg,max(Wind_Velocity_Mtr_Sec) as max from wind group by Station_ID").show()

+----------+------------------+-----+
|Station_ID|               avg|  max|
+----------+------------------+-----+
|      SF37| 2.260403505500663|7.079|
|      SF15|1.8214145677504483| 7.92|
|      SF04| 2.300981748124102| 9.92|
|      SF17|0.5183500253485376|5.767|
|      SF18|2.2202234391695437| 9.85|
|      SF36| 2.464172530911313| 9.71|
+----------+------------------+-----+



In [14]:
pan=df.toPandas()

In [15]:
pan.head()

,Station_ID,Station_Name,Location_Label,Interval_Minutes,Interval_End_Time,Wind_Velocity_Mtr_Sec,Wind_Direction_Variance_Deg,Wind_Direction_Deg,Ambient_Temperature_Deg_C,Global_Horizontal_Irradiance
0,SF15,"Warnerville Switchyard, Oakdale",Warnerville,5,2015-01-5? 00:05,1.628,8.1,148.5,0.92,0.061
1,SF15,"Warnerville Switchyard, Oakdale",Warnerville,5,2015-01-5? 00:10,1.519,9.4,151.1,0.717,0.064
2,SF15,"Warnerville Switchyard, Oakdale",Warnerville,5,2015-01-5? 00:15,1.482,8.7,142.7,0.627,0.059
3,SF15,"Warnerville Switchyard, Oakdale",Warnerville,5,2015-01-5? 00:20,1.985,6.895,141.8,0.5,0.062
4,SF15,"Warnerville Switchyard, Oakdale",Warnerville,5,2015-01-5? 00:25,1.903,8.64,144.7,0.727,0.062
